In [3]:
import glob
import nibabel
import pandas as pd
import numpy as np
import os

from collections import defaultdict
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook, tqdm
from joblib import Parallel, delayed
from IPython.core.debugger import set_trace

import torch
from torch import nn
from torch import autograd
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from utils import show_slices, check_patch, get_symmetric_value, pad_arrays, load, create_dicts

from multiprocessing import cpu_count
N_CPU = cpu_count()

In [4]:
labels_components = np.load('labels_info.npy', allow_pickle=True).item()
single_component_keys = {k for k,v in labels_components.items() if len(v['cc3d'][0]) == 2}

USE_GEOM_FEATURES = True
GEOM_FEATURES = ['thickness', 'sulc', 'curv']

root_label = '../fcd_data/normalized_label'
root_data = '../fcd_data/normalized_data/'
root_geom_features = '../fcd_data/preprocessed_data_anadezhda/'

paths_dict = create_dicts(root_label,
                         root_data,
                         root_geom_features, 
                         single_component_keys,
                         USE_GEOM_FEATURES, 
                         GEOM_FEATURES)

In [5]:
metadata = np.load('metadata.npy',allow_pickle=True).item()

In [9]:
AUGMENTATION_STEPS = 2
PATCH_SIZE=100
pad=PATCH_SIZE//2

In [11]:
for split_type, split_keys in metadata.items():
    for k in tqdm(split_keys):
        
        path_dict = paths_dict[k]
        brain_tensor, mask_tensor, label_tensor = load(path_dict) # float, bool, int
     
        X,Y,Z = mask_tensor.shape
        X_mean = X//2
        
        # get rid of a mid-brain
        thresh_mask = (np.arange(X) < (X_mean - pad)) | (np.arange(X) > (X_mean + pad))
        thresh_mask = np.tile(thresh_mask, (Y,Z,1)).transpose(2,0,1)
        mask_tensor = mask_tensor*(thresh_mask > 0)
        
        break
    break
#         ###########################
#         # CREATE RELEVANT INDEXES #
#         ###########################
#         xyz_grid = np.stack(np.meshgrid(np.arange(X), np.arange(Y), np.arange(Z), indexing='ij'), -1)
#         xyz_grid = xyz_grid[mask_tensor]

#         indexes_selected = Parallel(n_jobs=-1)(delayed(check_patch)(x,y,z,\
#                                                                     mask_tensor,\
#                                                                     label_tensor,\
#                                                                     pad) \
#                                                                for x,y,z in xyz_grid)

#         indexes_selected = list(filter(lambda x: x is not None, indexes_selected))
#         df = pd.DataFrame(indexes_selected)
#         df['is_train'] = split_type == 'train'
#         df['label'] = k
#         df.to_csv(f'../fcd_data/indexes_selected/{split_type}_{k}')

  0%|                                                                                                                                         | 0/69 [00:02<?, ?it/s]


In [44]:
np.unique(mask_tensor.astype(float))

array([0., 1.])

In [57]:
mask_tensor_torch.shape

torch.Size([1, 1, 241, 336, 283])

In [63]:
W = torch.ones(1,1,PATCH_SIZE,PATCH_SIZE,PATCH_SIZE, dtype=torch.float).to(DEVICE) 

KeyboardInterrupt: 

In [62]:
p_perc = torch.conv3d(input=mask_tensor_torch, weight=W, padding=pad) 
p_perc.shape

torch.Size([1, 1, 242, 337, 284])

In [53]:
p_perc = p_perc.detach().cpu().numpy()
l_sum = l_sum.detach().cpu().numpy()

In [55]:
p_perc.shape

(1, 1, 192, 287, 234)

In [51]:
l_sum.max()

tensor(7764., device='cuda:2')

In [50]:
# model = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True)